In [1]:
test_urls = ["http://203.64.136.248/webpacIndex.jsp",
             "https://libelis.tmu.edu.tw/webpacIndex.jsp",
             # "https://libholding.ntut.edu.tw/webpacIndex.jsp",  # 北科大：欄位名稱不是叫 ISBN
             "http://webpac.ntua.edu.tw/webpacIndex.jsp",
             "https://webpac.cute.edu.tw/webpacIndex.jsp",
             "http://lib.utaipei.edu.tw/webpac/webpacIndex.jsp",
             "http://webpac.ntub.edu.tw/webpacIndex.jsp",
             "http://libsearch.tcpa.edu.tw/webpacIndex.jsp",
             "http://192.192.231.232/webpacIndex.jsp",  # 中華科大：有些詳細書目裡面沒有表格
             # "http://webpac.knjc.edu.tw/webpacIndex.jsp",  # 康寧：不能以 ISBN 搜尋
             "http://webpac.cct.edu.tw/webpacIndex.jsp",
             "http://210.60.142.23/webpacIndex.jsp",
             "https://jinwenlib.just.edu.tw/webpacIndex.jsp",
             "http://hylib.chihlee.edu.tw/webpacIndex.jsp",
             "http://webpac.lib.vnu.edu.tw/webpacIndex.jsp",
             "https://library.uch.edu.tw/webpacIndex.jsp",
             "https://hylib.lib.must.edu.tw/webpacIndex.jsp",
             "https://hyweblib.nou.edu.tw/webpac/webpacIndex.jsp",
             "https://webpac.miaoli.gov.tw/webpacIndex.jsp",
             "http://120.106.11.155/webpacIndex.jsp",
             "http://libopac.jente.edu.tw/webpacIndex.jsp",
             "http://webpac.lib.ntcu.edu.tw/webpacIndex.jsp",
             "https://hylib.ntus.edu.tw/webpacIndex.jsp",
             "https://webpac.lib.thu.edu.tw/webpacIndex.jsp",
             "http://webpac.lib.pu.edu.tw/webpac/webpacIndex.jsp",
             "http://lib.webpac.ocu.edu.tw/webpacIndex.jsp",
             "https://book.ncue.edu.tw/webpacIndex.jsp",
             "http://library.ylccb.gov.tw/webpacIndex.jsp",
             "https://www.cycab.gov.tw/webpacIndex.jsp",
             "http://library.cabcy.gov.tw/webpacIndex.jsp",
             "http://hylib.nhu.edu.tw//webpacIndex.jsp",
             "http://webpac.cnu.edu.tw/webpacIndex.jsp",
             "http://hy.lib.feu.edu.tw/webpacIndex.jsp",
             "https://webpac2.csu.edu.tw/webpacIndex.jsp",
             "http://webpac.meiho.edu.tw/webpacIndex.jsp",
             "https://lib.niu.edu.tw/webpacIndex.jsp",
             "http://libils.fgu.edu.tw/webpacIndex.jsp",
             "http://hylib.lib.nttu.edu.tw/webpac/webpacIndex.jsp",
             "https://lib.nqu.edu.tw/webpacIndex.jsp"]

In [2]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.common.exceptions import NoSuchElementException, TimeoutException

import pandas as pd
from bs4 import BeautifulSoup
import time

my_options = Options()
my_options.add_argument("--incognito")  # 開啟無痕模式
my_options.add_argument("--start-maximized")  # 視窗最大化
# my_options.add_argument("--headless")  # 不開啟實體瀏覽器
capabilities = DesiredCapabilities.CHROME
capabilities["pageLoadStrategy"] = "none"

In [21]:
def test(org, url, ISBN):
    # url = "http://libils.fgu.edu.tw/webpacIndex.jsp"
    # ISBN = "9789573317241"  # 哈利波特(1)：神秘的魔法石

    test_url = url[:]  # 測試用，輸出「現在測試的網頁」
    test_start = time.time()  # 測試用，計算程式進行時間

    # 初始化 driver；設定 explicit waits
    driver = webdriver.Chrome(options=my_options, desired_capabilities=capabilities)
    wait = WebDriverWait(driver, 30)  # 最長等待時間：30 秒；查找元素的間隔時間：0.5秒（default）

    # 進入＂搜尋主頁＂
    driver.get(url)

    # 等待定位＂下拉式選單＂，選擇 ISBN
    search_field = wait.until(
        EC.presence_of_element_located((By.NAME, "search_field"))
    )
    select = Select(search_field)
    select.select_by_value("ISBN")

    org = driver.title

    # 等待定位＂搜尋欄＂，輸入 ISBN
    search_input = wait.until(
        EC.presence_of_element_located((By.NAME, "search_input"))
    )
    search_input.send_keys(ISBN)
    search_input.submit()

    # 依據有沒有＂div.mainCon＂，來判斷搜尋結果是「＂一筆＂」還是「＂零筆＂或＂多筆＂」
    try:  # 3 秒內可以找到元素，表示只有＂一筆＂搜尋結果
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "div.mainCon"))
        )

        # 依據有沒有＂table.order＂，來判斷該頁面＂有＂表格還是＂沒有＂表格
        # 例如：「http://192.192.231.232/bookDetail.do?id=260965&resid=188809782&nowid=3」，
        # 有頁面但沒有表格
        try:
            WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "table.order"))
            )
            tgt = pd.read_html(driver.page_source, encoding="utf-8")[-2]  # 定位＂表格＂，爬取內容
            tgt["圖書館"], tgt["連結"] = org, driver.current_url

            driver.close()
            test_end = time.time()  # 測試用，計算程式進行時間
            print(f"程式執行成功，執行時間：{test_end - test_start:7.4f}秒，在「{url}」")  # 測試用，計算程式進行時間
            return tgt.fillna("")
        except:  # 5 秒內＂找不到＂ table.order，表示該頁面有表格
            print(f"該頁面沒有表格：「{driver.current_url}」")
    except (NoSuchElementException, TimeoutException):  # 3 秒內＂找不到＂元素，，表示有＂多筆＂搜尋結果
        root_url = url.replace(url.split("/")[-1], "")  # root_url 用來組成不同的＂詳細書目＂的網址

        tgt = []
        # 定位＂內嵌框架＂，切換至此內部，找尋多個＂詳細書目的網址＂
        iframe = wait.until(
            EC.presence_of_element_located((By.ID, "leftFrame"))
        )
        driver.switch_to.frame(iframe)
        soup = BeautifulSoup(driver.page_source, "html.parser")

        # 組合網址
        tgt_urls = []
        for anchor in soup.find_all("a", "bookname"):
            tgt_urls.append(root_url + anchor["href"])

        # 進入不同的＂詳細書目＂
        for urlx in tgt_urls:
            driver.get(urlx)

            # 依據有沒有＂table.order＂，來判斷該頁面＂有＂表格還是＂沒有＂表格
            try:
                WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "table.order"))
                )
                sec_tgt = pd.read_html(driver.page_source, encoding="utf-8")[-2]  # 定位＂表格＂，爬取內容
                sec_tgt["圖書館"], sec_tgt["連結"] = org, driver.current_url
                tgt.append(sec_tgt)
            except:
                print(f"該頁面沒有表格：「{driver.current_url}」")

        driver.close()
        if tgt != []:
            tgt = pd.concat(tgt, axis=0, ignore_index=True)
            test_end = time.time()  # 測試用，計算程式進行時間
            print(f"程式執行成功，執行時間：{test_end - test_start:7.4f}秒，在「{url}」")  # 測試用，計算程式進行時間
            return tgt.fillna("")
    except Exception as e:
        test_end = time.time()  # 測試用，計算程式進行時間
        print("="*100)
        print(f"程式執行失敗，執行時間：{test_end - test_start:7.4f}秒，在「{org}」")
        print(f"網址為：「{url}」")
        print(f"錯誤訊息為：「{e}」")
        print("="*100)

In [4]:
df_final = []
ISBN = "9789573317241"  # 哈利波特(1)：神秘的魔法石
for test_url in test_urls:
    df_final.append(test("測試圖書館", test_url, ISBN))

程式執行成功，執行時間： 5.3172秒，在「http://203.64.136.248/webpacIndex.jsp」
程式執行成功，執行時間： 5.4690秒，在「https://libelis.tmu.edu.tw/webpacIndex.jsp」
程式執行成功，執行時間： 4.1899秒，在「http://webpac.ntua.edu.tw/webpacIndex.jsp」
該頁面沒有表格：「https://webpac.cute.edu.tw/bookDetail.do?id=83506&Lflag=1」
程式執行成功，執行時間：13.9379秒，在「https://webpac.cute.edu.tw/webpacIndex.jsp」
程式執行成功，執行時間：21.1798秒，在「http://lib.utaipei.edu.tw/webpac/webpacIndex.jsp」
程式執行成功，執行時間： 4.1415秒，在「http://webpac.ntub.edu.tw/webpacIndex.jsp」
程式執行成功，執行時間：13.4292秒，在「http://libsearch.tcpa.edu.tw/webpacIndex.jsp」
該頁面沒有表格：「http://192.192.231.232/bookDetail.do?id=260965&resid=188809790&nowid=3」
程式執行成功，執行時間：19.2081秒，在「http://192.192.231.232/webpacIndex.jsp」
程式執行成功，執行時間： 3.5782秒，在「http://webpac.cct.edu.tw/webpacIndex.jsp」
程式執行成功，執行時間： 4.4127秒，在「http://210.60.142.23/webpacIndex.jsp」
程式執行成功，執行時間： 5.7234秒，在「https://jinwenlib.just.edu.tw/webpacIndex.jsp」
程式執行成功，執行時間： 5.6716秒，在「http://hylib.chihlee.edu.tw/webpacIndex.jsp」
程式執行成功，執行時間：10.2352秒，在「http://webpac.lib.vnu.edu.tw/we

In [23]:
final_output = pd.concat(df_final, axis=0, ignore_index=True)
final_output.fillna("")

,#,條碼號,館藏位置,索書號,資料類型,目前狀態 / 到期日,架號,附件,預約,圖書館,...,狀態,館藏室,館藏地,特定用途,館藏地/區域,特藏類型,圖書狀況 / 到期日,註記,編號,館藏地/館藏室
0,1.0,99502,中文書櫃,873.57 857 v.1,一般圖書/一般,書在館,,,不可預約,經國管理暨健康學院 圖書館,...,,,,,,,,,,
1,2.0,99505,中文書櫃,873.57 857 v.1 c.4,一般圖書/一般,架上遺失,,,不可預約,經國管理暨健康學院 圖書館,...,,,,,,,,,,
2,3.0,99503,中文書櫃,873.57 857 v.1 c.2,一般圖書/一般,書在館,,,不可預約,經國管理暨健康學院 圖書館,...,,,,,,,,,,
3,4.0,99504,第三書庫,873.57 857 v.1 c.3,一般圖書/一般,不可外借,,,不可預約,經國管理暨健康學院 圖書館,...,,,,,,,,,,
4,1.0,C0090480,,873.57 6014-1 89,中日韓圖書/一般,書在館,,,不可預約,臺北醫學大學圖書館,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,6.0,,,＋ 846 6014,一般圖書(兒童)/一般,可預約調借,,,/ 0人預約,國立臺東大學 館藏查詢系統,...,,,,,,,,,,
142,1.0,E227675,,873.57 857 89 c.4,圖書/一般,書在館,,,不可預約,國立金門大學圖書館,...,,,,,,,,,,
143,2.0,E219874,,873.57 857 89 c.3,圖書/一般,書在館,,,不可預約,國立金門大學圖書館,...,,,,,,,,,,
144,3.0,E178789,,873.57 857 89 c.2,圖書/一般,書在館,,,不可預約,國立金門大學圖書館,...,,,,,,,,,,


In [8]:
final_output.to_csv(f"檢查.csv", encoding="big5", errors="ignore")

In [38]:
def organize_columns(df):
    # 處理 column 2：館藏地
    c2 = ["館藏地/室", "館藏室", "館藏地/館藏室", "館藏地", "典藏館", "館藏位置", "館藏地/區域"]
    df["c2"] = ""
    for c in c2:
        try:
            df["c2"] += df[c]
        except:
            pass
    
    # 處理 column 3：索書號
    c3 = ["索書號", "索書號/期刊合訂本卷期", "索書號 / 部冊號"]
    df["c3"] = ""
    for c in c3:
        try:
            df["c3"] += df[c]
        except:
            pass

    # 處理 column 4：館藏狀態
    c4 = ["狀態/到期日", "目前狀態 / 到期日", "館藏狀態", "處理狀態", '狀態 (說明)', '館藏現況 說明', "目前狀態/預計歸還日期", "圖書狀況 / 到期日"]
    df["c4"] = ""
    for c in c4:
        try:
            df["c4"] += df[c]
        except:
            pass
    
    # 直接生成另一個 DataFrame
    test = pd.DataFrame()
    test["圖書館"] = df["圖書館"]
    test["館藏地"] = df["c2"]
    test["索書號"] = df["c3"]
    test["館藏狀態"] = df["c4"]
    test["連結"] = df["連結"]
    
    return test

In [45]:
import pandas.io.formats.excel

In [53]:
def set_excel(df, directory):
    # B｢圖書館｣、C「館藏地」、D「索書號」、E「館藏狀態」、F「連結」
    pandas.io.formats.excel.header_style = None  # 標題格式清除
    writer = pd.ExcelWriter(directory)
    df.to_excel(writer, sheet_name="搜尋結果")

    workbook1 = writer.book
    worksheets = writer.sheets
    worksheet1 = worksheets["搜尋結果"]
    
    # 測試
    cell_format = workbook1.add_format({
        "font_name": "微軟正黑體",
        "font_size": 16,
        "align": "left",
#         "border": 80,
    })
    worksheet1.set_column("B:F", 40, cell_format)
    
    # 設定單元格的寬度
#     worksheet1.set_column("B:F", 35)

    writer.save()
    print("爬取完成")

In [54]:
wwww = organize_columns(final_output.fillna(""))
# wwww.to_csv(f"檢查.csv", encoding="big5", errors="ignore")
set_excel(wwww, f"tttttt.xlsx")
wwww

爬取完成


,圖書館,館藏地,索書號,館藏狀態,連結
0,經國管理暨健康學院 圖書館,中文書櫃,873.57 857 v.1,書在館,http://203.64.136.248/bookDetail.do?id=57928
1,經國管理暨健康學院 圖書館,中文書櫃,873.57 857 v.1 c.4,架上遺失,http://203.64.136.248/bookDetail.do?id=57928
2,經國管理暨健康學院 圖書館,中文書櫃,873.57 857 v.1 c.2,書在館,http://203.64.136.248/bookDetail.do?id=57928
3,經國管理暨健康學院 圖書館,第三書庫,873.57 857 v.1 c.3,不可外借,http://203.64.136.248/bookDetail.do?id=57928
4,臺北醫學大學圖書館,北醫總館/2F中文圖書區,873.57 6014-1 89,書在館,https://libelis.tmu.edu.tw/bookDetail.do?id=95910
...,...,...,...,...,...
141,國立臺東大學 館藏查詢系統,知本總館/3F兒童文學書庫,＋ 846 6014,可預約調借,http://hylib.lib.nttu.edu.tw/webpac/bookDetail...
142,國立金門大學圖書館,國立金門大學各系所/金沙校區學生宿舍,873.57 857 89 c.4,書在館,https://lib.nqu.edu.tw/bookDetail.do?id=9668
143,國立金門大學圖書館,國立金門大學圖書館/圖書館,873.57 857 89 c.3,書在館,https://lib.nqu.edu.tw/bookDetail.do?id=9668
144,國立金門大學圖書館,國立金門大學圖書館/圖書館,873.57 857 89 c.2,書在館,https://lib.nqu.edu.tw/bookDetail.do?id=9668


In [ ]:
final_output.to_csv(f"{org}.csv", encoding="big5", errors="ignore")

In [ ]:
driver = webdriver.Chrome(options=my_options, desired_capabilities=capabilities)
wait = WebDriverWait(driver, 30)  # 最長等待時間：30 秒；查找元素的間隔時間：0.5秒（default）
driver.get("http://192.192.231.232/webpacIndex.jsp")

In [ ]:
search_field = wait.until(
    EC.presence_of_element_located((By.NAME, "search_field"))
)
select = Select(search_field)
select.select_by_value("ISBN")

# 定位＂搜尋欄＂，輸入 ISBN
search_input = wait.until(
    EC.presence_of_element_located((By.NAME, "search_input"))
)
search_input.send_keys(ISBN)
search_input.submit()